Data-wrangling Sample


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


After successful mount, ctrl + click the drive. Navigate the shared folder, and look for the file 'sna_raw_data.csv', and then copy the path.

Then, paste the path to the next line of code

In [ ]:
filename = '/content/drive/MyDrive/VLL Wall Post LAK/sna_raw_data.csv'

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(filename, encoding='utf-8')
#df.head
df.columns

Index(['post_id', 'useraccount_id', 'thread_id', 'post/comment text',
       'timestamp', 'karma points awarded', 'post type', 'role'],
      dtype='object')

In [ ]:
row_headers = ['post_id','useraccount_id', 'thread_id', 'post/comment text', 'timestamp', 'karma points awarded', 'post type', 'role']
df = pd.read_csv(filename, names=row_headers, skiprows=1)
reordered_columns = ['timestamp',  'post_id', 'useraccount_id', 'thread_id', 'post type', 'role', 'karma points awarded', 'post/comment text']
df = df[reordered_columns]
df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)
df = df.sort_values(by='timestamp')
#df.to_csv('sna_preprocessed.csv', index=False, header=True)

In [ ]:
print(df.head())
print (df.columns)

            timestamp  ...                                  post/comment text
1 2018-08-01 13:00:00  ...         hi Lucas!  what can I help you with today?
2 2018-08-01 13:02:00  ...  Topic 9: Solution Sets to Equations with Two V...
3 2018-08-01 13:05:00  ...  do you need help with this problem or have a q...
4 2018-08-01 13:21:00  ...  hi Jeevika!  where do you see the SAT prep but...
5 2018-08-01 13:23:00  ...  when you go to the algebra 2 videos at the top...

[5 rows x 8 columns]
Index(['timestamp', 'post_id', 'useraccount_id', 'thread_id', 'post type',
       'role', 'karma points awarded', 'post/comment text'],
      dtype='object')


In [ ]:
df.shape

(171990, 8)

In [ ]:
print(df['post_id'].nunique())
print(df['thread_id'].nunique())
print(df['useraccount_id'].nunique())
print(df['role'].nunique())
print(df['karma points awarded'].nunique())
print(df['post type'].nunique())

print(df['role'].unique())
print(df['karma points awarded'].unique())
print(df['post type'].unique())

171990
14383
4720
4
4
2
['Study expert' 'Student' 'Teacher' 'Study expert/Guide']
[  0 100 200 500]
['comment' 'post']


From the prints above, there are a total of 171990 rows which is the number of posts. 14383 is the total number of unique threads, and there are a total of 4720 unique users. The users are also either a student expert, student, teacher, or a study expert/guide. The karma points were also either 0, 100, 200, or 500. Lastly, each post is either a comment or a post.

In the following lines of code, I will generate a csv file based on specific groupings.

1st Grouping: Group by users, and count their respective comments and posts

In [ ]:
df_users = df.groupby('useraccount_id')['post type'].value_counts().unstack().fillna(0)
#df_users.to_csv('sna_processed_data-users.csv', header=True)

In [ ]:
df_users.describe()

post type,comment,post
count,4720.000000,4720.000000
mean,34.207203,2.231356
std,269.896249,7.780515
min,0.000000,0.000000
25%,1.000000,0.000000
50%,3.000000,1.000000
75%,10.000000,2.000000
max,9340.000000,221.000000


In [ ]:
df[df['useraccount_id'] == 5272649]

2nd Grouping: Group by threads, and count the number of comments and posts

In [ ]:
df_threads = df.groupby('thread_id')['post type'].value_counts().unstack().fillna(0)
#df_threads.to_csv('sna_processed_data-threads.csv', header=True)

In [ ]:
print(df_threads.head())
df_threads.describe()

post type  comment  post
thread_id               
221216         1.0   0.0
366837         6.0   0.0
407299         1.0   0.0
511996         6.0   0.0
664724         1.0   0.0


post type,comment,post
count,14383.000000,14383.000000
mean,11.225614,0.732253
std,13.026648,0.442800
min,0.000000,0.000000
25%,3.000000,0.000000
50%,7.000000,1.000000
75%,14.000000,1.000000
max,245.000000,1.000000


3rd Grouping: Group by thread, and count the number of users in each thread (Columns are the number of unique users, and the rows are the unique number of threads, so this is a very large file and will take a while to run)

In [ ]:
df_threads_users = df.groupby('thread_id')['useraccount_id'].value_counts().unstack().fillna(0)
df_threads_users.to_csv('sna_processed_data-threads-users.csv', header=True)

In [ ]:
print(df_threads_users.head())

4th Grouping: Group by thread, and count number of different types of users in each thread

In [ ]:
df_threads_roles = df.groupby('thread_id')['role'].value_counts().unstack().fillna(0)
df_threads_roles.to_csv('sna_processed_data-threads-roles.csv', header=True)

In [ ]:
df_threads_roles.describe()

role,Student,Study expert,Study expert/Guide,Teacher
count,14383.000000,14383.000000,14383.000000,14383.000000
mean,8.852187,3.095808,0.001738,0.008135
std,10.443702,3.616853,0.041656,0.158223
min,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,0.000000,0.000000
50%,6.000000,2.000000,0.000000,0.000000
75%,12.000000,4.000000,0.000000,0.000000
max,228.000000,65.000000,1.000000,9.000000


In [ ]:
print(df_threads_roles.head())

role       Student  Study expert  Study expert/Guide  Teacher
thread_id                                                    
221216         1.0           0.0                 0.0      0.0
366837         6.0           0.0                 0.0      0.0
407299         1.0           0.0                 0.0      0.0
511996         6.0           0.0                 0.0      0.0
664724         0.0           1.0                 0.0      0.0


In this section, the network graph will be created using the networkx module. We will also need sklearn to normalize the weights of the edges of the network. Itertools will also be used for easier generation of combinations when we are adding edges to the network.

For the network to be generated, we will only be needing the useraccount_id and thread_id columns

In [ ]:
df = pd.read_csv('sna_preprocessed.csv')
df = df[['post/comment text', 'thread_id', 'useraccount_id']]

In [ ]:
print(df.head())

                                   post/comment text  thread_id  useraccount_id
0         hi Lucas!  what can I help you with today?    2578176         7995389
1  Topic 9: Solution Sets to Equations with Two V...    2578178         4875726
2  do you need help with this problem or have a q...    2578178         7995389
3  hi Jeevika!  where do you see the SAT prep but...    2578180         7995389
4  when you go to the algebra 2 videos at the top...    2578180         5335377


Next, lets group the dataframe by thread_id and then transform the useraccount_id into a list of unique useraccount_ids per unique thread.

In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['would', 'uhmm'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df_threads = df.groupby('thread_id')['useraccount_id'].apply(set).reset_index(name='useraccount_ids')

In [ ]:
df.post_or_comment_text = df.post_or_comment_text.str.replace("[^A-Za-z ]", " ")
#df.post_or_comment_text = str(df.post_or_comment_text)
#df.post_or_comment_text = df.post_or_comment_text.apply(cleaning)

In [ ]:
# Cleaning Function
def cleaning(string):
    string = string.lower()
    string = string.replace('àö', '')
    string = string.replace('çå', '')
    string = string.replace('ç', '')
    string = string.replace('å', '')
    string = string.replace('äôs', '')
    string = string.replace('çç', '')
    string = string.replace('äôt', '')
    string = string.lower()
   
    from nltk.stem import WordNetLemmatizer
    import nltk
    words = nltk.word_tokenize(string)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(i) for i in words]
    new_words = []
    for i in words:
        if i.isdigit():
            new_words.append('NUMBER')
        else:
            new_words.append(i)
    return ' '.join(new_words)

In [ ]:
df_threads_text = df.groupby('thread_id')['post/comment text'].apply(set).reset_index(name='post/comment texts')

In [ ]:
df_threads_userid = df.groupby('thread_id')['useraccount_id'].apply(set).reset_index(name='useraccount_ids')

In [ ]:
print(df_threads_text.head())

   thread_id                                 post/comment texts
0     221216  {This may not be related to Algebra and all, b...
1     366837  {Simon, this post was already answered a long ...
2     407299               {THANK YOU SO MUCH! You are amazing}
3     511996  {yes they are all polynomials, yeah they are a...
4     664724  {this post is answered and don't use that lang...


In [ ]:
df_threads_userid.to_csv('sna_processed_data-users-in-thread.csv', header=True)

In [ ]:
print(df_threads_userid.head())


   thread_id                                useraccount_ids
0     221216                                      {2622189}
1     366837                    {2565621, 6763421, 8573191}
2     407299                                      {1800818}
3     511996  {4582984, 4516785, 5357979, 3175581, 3237182}
4     664724                                      {7821006}


The next line will discard rows which has only one useraccount_id. If a thread_id only has one useraccount_id, then it cannot be associated with any other user in any way.

In [ ]:
df_threads_text['length'] = df_threads_text['post/comment texts'].apply(len)
df_threads_text = df_threads_text[df_threads_text['length'] > 1]

The data will now look like this:

In [ ]:
print(df_threads_text.head())
df_threads_text.to_csv('threads_texts_0917.csv', header=True)

   thread_id                                 post/comment texts  length
1     366837  {Simon, this post was already answered a long ...       6
3     511996  {yes they are all polynomials, yeah they are a...       6
5     817211                 {I put like for everyone., Thanks}       2
6     960330  {Go to the top and find videos and more. you s...       5
7     979597  {Hm, i honesly do not how to do but i know the...       2
